# sktime Pipelines for Forecasting

Pipelines let you chain preprocessing, modeling, and evaluation into a single object that can be tuned
and backtested without leakage. This notebook shows common sktime patterns: transformed-target
pipelines, reduction to regression, and systematic backtesting with Plotly visuals.


## Pipeline anatomy

- **Transformers**: per-series transforms like Box-Cox, detrending, or deseasonalizing.
- **Forecasters**: classical models (Theta, ARIMA), ML reduction, or deep models.
- **Evaluation**: temporal splits and backtesting with expanding or sliding windows.

In sktime, `TransformedTargetForecaster` is the most direct way to chain transforms on the target
series before applying a forecaster. For feature-based ML, `make_reduction` turns a regressor into a
forecaster with a sliding window.


## Mathematical view

Let transformations $T_1,\dots,T_k$ act on the target series $y_t$. A pipeline applies them in order,
then fits a forecaster $F$ to the transformed series:

$$z_t = T_k(\dots T_1(y_t))$$
$$\hat{z}_{t+h} = F(z_{1:t})$$
$$\hat{y}_{t+h} = T_1^{-1}(\dots T_k^{-1}(\hat{z}_{t+h}))$$

Fitting the transformations **inside** the pipeline is what prevents leakage during backtesting.


In [ ]:
import plotly.graph_objects as go

steps = ["Raw series", "Box-Cox", "Deseasonalize", "Forecaster", "Forecasts"]
fig = go.Figure()

for i, label in enumerate(steps):
    x0 = i * 1.25
    x1 = x0 + 1.0
    fig.add_shape(
        type="rect",
        x0=x0,
        x1=x1,
        y0=0,
        y1=0.6,
        line=dict(color="#1f77b4", width=2),
        fillcolor="rgba(31, 119, 180, 0.15)",
    )
    fig.add_annotation(x=(x0 + x1) / 2, y=0.3, text=label, showarrow=False)
    if i < len(steps) - 1:
        fig.add_annotation(x=x1 + 0.1, y=0.3, text="->", showarrow=False, font=dict(size=18))

fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(
    title="Forecasting pipeline sketch",
    height=220,
    margin=dict(l=20, r=20, t=50, b=20),
)
fig

## Synthetic monthly series

We will create a small seasonal series with trend and noise. This keeps the example reproducible and
clear for plotting.


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

np.random.seed(7)

n_periods = 120
index = pd.period_range("2015-01", periods=n_periods, freq="M")
trend = 0.2 * np.arange(n_periods)
seasonal = 4 * np.sin(2 * np.pi * np.arange(n_periods) / 12)
noise = np.random.normal(0, 1.0, n_periods)

series = pd.Series(20 + trend + seasonal + noise, index=index, name="y")
series.head()


In [ ]:
idx = series.index.to_timestamp()
fig = go.Figure()
fig.add_trace(go.Scatter(x=idx, y=series, mode="lines", name="y", line=dict(color="#1f77b4")))
fig.update_layout(title="Synthetic monthly series", xaxis_title="time", yaxis_title="value", height=420)
fig

## Train/test split and forecasting horizon

A forecasting horizon $H = \{h_1,\dots,h_k\}$ defines which steps ahead we predict.
We will use an absolute horizon based on the test index for clarity.


In [ ]:
from sktime.forecasting.model_selection import temporal_train_test_split, ForecastingHorizon

y_train, y_test = temporal_train_test_split(series, test_size=24)
fh = ForecastingHorizon(y_test.index, is_relative=False)

len(y_train), len(y_test)


## Example 1: Transformed-target pipeline

We chain a Box-Cox transform and deseasonalizer before forecasting with the Theta method. The entire
sequence is a single estimator, so backtesting treats it as one unit.


In [ ]:
from sktime.transformations.series.boxcox import BoxCoxTransformer
from sktime.transformations.series.detrend import Deseasonalizer
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.performance_metrics.forecasting import mean_absolute_error, mean_absolute_percentage_error

pipe = TransformedTargetForecaster(
    steps=[
        ("boxcox", BoxCoxTransformer()),
        ("deseasonalize", Deseasonalizer(sp=12, model="multiplicative")),
        ("forecaster", ThetaForecaster(sp=12)),
    ]
)

pipe.fit(y_train)
y_pred = pipe.predict(fh)

mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"MAE: {mae:.3f}")
print(f"MAPE: {mape * 100:.2f}%")


In [ ]:
train_idx = y_train.index.to_timestamp()
test_idx = y_test.index.to_timestamp()
pred_idx = y_pred.index.to_timestamp()

fig = go.Figure()
fig.add_trace(go.Scatter(x=train_idx, y=y_train, mode="lines", name="Train", line=dict(color="#1f77b4")))
fig.add_trace(go.Scatter(x=test_idx, y=y_test, mode="lines", name="Test", line=dict(color="#ff7f0e")))
fig.add_trace(go.Scatter(x=pred_idx, y=y_pred, mode="lines", name="Forecast", line=dict(color="#2ca02c")))
fig.update_layout(title="Pipeline forecast vs actual", xaxis_title="time", yaxis_title="value", height=420)
fig

## Backtesting with expanding windows

Backtesting evaluates a pipeline over multiple cutoffs. The expanding window strategy starts with a
minimum training size and grows the training window at each split.


In [ ]:
from sktime.forecasting.model_selection import ExpandingWindowSplitter
from sktime.forecasting.model_evaluation import evaluate

cv = ExpandingWindowSplitter(initial_window=60, step_length=12, fh=[1, 2, 3, 6, 12])
results = evaluate(
    pipe,
    series,
    cv=cv,
    strategy="refit",
    scoring=mean_absolute_error,
)

results.head()


In [ ]:
metric_col = [col for col in results.columns if col.startswith("test_")][0]
cutoff = results["cutoff"].astype(str)

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=cutoff,
        y=results[metric_col],
        marker_color="#ff7f0e",
    )
)
fig.update_layout(
    title=f"Backtesting {metric_col.replace('test_', '')} by cutoff",
    xaxis_title="cutoff",
    yaxis_title=metric_col,
    height=380,
)
fig

## Example 2: Reduction to regression

For ML pipelines, `make_reduction` wraps a standard regressor to forecast using lagged windows. This
is a simple path to tree-based or linear models without manual feature engineering.


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sktime.forecasting.compose import make_reduction

reduction_forecaster = make_reduction(
    RandomForestRegressor(n_estimators=200, random_state=7),
    strategy="recursive",
    window_length=24,
)

reduction_forecaster.fit(y_train)
y_pred_ml = reduction_forecaster.predict(fh)
mae_ml = mean_absolute_error(y_test, y_pred_ml)

print(f"Reduction forecaster MAE: {mae_ml:.3f}")


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_idx, y=y_test, mode="lines", name="Test", line=dict(color="#ff7f0e")))
fig.add_trace(go.Scatter(x=pred_idx, y=y_pred, mode="lines", name="Theta pipeline", line=dict(color="#2ca02c")))
fig.add_trace(
    go.Scatter(
        x=y_pred_ml.index.to_timestamp(),
        y=y_pred_ml,
        mode="lines",
        name="Reduction (RF)",
        line=dict(color="#9467bd", dash="dash"),
    )
)
fig.update_layout(title="Comparing pipeline vs reduction forecasts", xaxis_title="time", yaxis_title="value", height=420)
fig

## Practical checklist

- Choose transforms that match the data (variance stabilization, seasonality, trend).
- Fit transforms **inside** the pipeline so backtesting avoids leakage.
- Use expanding windows for stable processes; sliding windows for non-stationary regimes.
- Track multiple metrics (MAE, MAPE, MASE) to see sensitivity to scale and outliers.


## Further reading

- sktime user guide and forecasting tutorials (pipelines, reduction, backtesting)
- scikit-learn pipeline patterns for transformers + estimators
